In [1]:
from collections import deque
import heapq
import random

class State:
    def __init__(self, puzzle, parent=None, action=None, cost=0):
        """
        Class to represent the state of the puzzle.
        """
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.cost = cost
        self.manhattan = self.calculate_manhattan()

    def __lt__(self, other):
        """
        Method to compare states for priority queue sorting.
        """
        return (self.cost + self.manhattan) < (other.cost + other.manhattan)

    def calculate_manhattan(self):
        """
        Method to calculate the Manhattan distance of the current state.
        """
        total_dist = 0
        for i in range(4):
            for j in range(4):
                if self.puzzle[i][j] != 0:
                    target_x, target_y = divmod(self.puzzle[i][j] - 1, 4)
                    total_dist += abs(i - target_x) + abs(j - target_y)
        return total_dist

class Game:
    def is_goal(self, state):
        """
        Checks if the current state is the goal state.
        """
        goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
        return state.puzzle == goal_state

    def get_possible_actions(self, state):
        """
        Returns possible actions for the current state.
        """
        actions = []
        blank_x, blank_y = self.find_blank(state.puzzle)
        if blank_x > 0:
            actions.append("down")
        if blank_x < 3:
            actions.append("up")
        if blank_y > 0:
            actions.append("right")
        if blank_y < 3:
            actions.append("left")
        return actions

    def get_next_state(self, state, action):
        """
        Returns the next state after taking a specific action.
        """
        blank_x, blank_y = self.find_blank(state.puzzle)
        new_puzzle = [list(row) for row in state.puzzle]
        if action == "up":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x + 1][blank_y] = new_puzzle[blank_x + 1][blank_y], new_puzzle[blank_x][blank_y]
        elif action == "down":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x - 1][blank_y] = new_puzzle[blank_x - 1][blank_y], new_puzzle[blank_x][blank_y]
        elif action == "left":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x][blank_y + 1] = new_puzzle[blank_x][blank_y + 1], new_puzzle[blank_x][blank_y]
        elif action == "right":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x][blank_y - 1] = new_puzzle[blank_x][blank_y - 1], new_puzzle[blank_x][blank_y]
        return State(new_puzzle, state, action, state.cost + 1)

    def find_blank(self, puzzle):
        """
        Finds the coordinates of the blank tile in the puzzle.
        """
        for i in range(4):
            for j in range(4):
                if puzzle[i][j] == 0:
                    return i, j

class Heuristics:
    @staticmethod
    def manhattan_distance(state):
        """
        Computes the Manhattan distance heuristic for a specific state.
        """
        total_dist = 0
        for i in range(4):
            for j in range(4):
                if state.puzzle[i][j] != 0:
                    target_x, target_y = divmod(state.puzzle[i][j] - 1, 4)
                    total_dist += abs(i - target_x) + abs(j - target_y)
        return total_dist

    @staticmethod
    def misplaced_tiles(state):
        """
        Computes the number of misplaced tiles heuristic for a specific state.
        """
        count = 0
        goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
        for i in range(4):
            for j in range(4):
                if state.puzzle[i][j] != goal_state[i][j]:
                    count += 1
        return count

class Agent:
    def __init__(self, game):
        """
        Initializes the agent with the game instance.
        """
        self.game = game

    def solve_BFS(self, initial_state):
        """
        Solves the puzzle using the Breadth-First Search algorithm.
        """
        queue = deque([initial_state])
        visited_states = set()
        level = 1
        nodes_explored = 0
        BFS_data = []

        while queue:
            level_size = len(queue)
            BFS_data.append([level, level_size])

            for _ in range(level_size):
                current_state = queue.popleft()
                current_state_tuple = tuple(map(tuple, current_state.puzzle))

                if current_state_tuple in visited_states:
                    continue

                visited_states.add(current_state_tuple)
                nodes_explored += 1

                if self.game.is_goal(current_state):
                    return  BFS_data

                possible_actions = self.game.get_possible_actions(current_state)
                if not possible_actions:
                    return

                for action in possible_actions:
                    next_state = self.game.get_next_state(current_state, action)
                    next_state_tuple = tuple(map(tuple, next_state.puzzle))
                    if next_state_tuple not in visited_states:
                        queue.append(next_state)

            level += 1

        return

    def solve_Astar_manhattan(self, initial_state):
        """
        Solves the puzzle using the A* algorithm with the Manhattan distance heuristic.
        """
        visited = set()
        steps_data = []
        priority_queue = [(0, initial_state)]

        steps_data.insert(0, {
            "manhattan_distance": Heuristics.manhattan_distance(initial_state),
            "state": initial_state.puzzle,
            "action": None,
            "parent": None,
            "moves": 0
        })

        while priority_queue:
            current_cost, current_state = heapq.heappop(priority_queue)

            visited.add(tuple(map(tuple, current_state.puzzle)))

            if self.game.is_goal(current_state):
                return steps_data

            for action in self.game.get_possible_actions(current_state):
                next_state = self.game.get_next_state(current_state, action)
                next_state_tuple = tuple(map(tuple, next_state.puzzle))

                if next_state_tuple not in visited:
                    next_manhattan = Heuristics.manhattan_distance(next_state)
                    next_cost = current_state.cost + 1

                    heapq.heappush(priority_queue, (next_cost + next_manhattan, next_state))
                    steps_data.append({
                        "manhattan_distance": next_manhattan,
                        "state": next_state.puzzle,
                        "action": action,
                        "parent": current_state.puzzle,
                        "moves": next_cost
                    })

        return None

    def solve_Astar_misplaced(self, initial_state):
        """
        Solves the puzzle using the A* algorithm with the misplaced tiles heuristic.
        """
        priority_queue = [(Heuristics.misplaced_tiles(initial_state), initial_state)]
        visited_states = set()
        steps_data = []

        while priority_queue:
            _, current_state = heapq.heappop(priority_queue)
            current_state_tuple = tuple(map(tuple, current_state.puzzle))

            if current_state_tuple in visited_states:
                continue

            visited_states.add(current_state_tuple)

            if self.game.is_goal(current_state):
                return steps_data

            count = Heuristics.misplaced_tiles(current_state)

            if not steps_data or count < steps_data[-1]["misplaced_tiles"]:
                steps_data.append({
                    "misplaced_tiles": count,
                    "state": current_state.puzzle,
                    "action": None,
                    "parent": current_state.parent.puzzle if current_state.parent else None
                })

            possible_actions = self.game.get_possible_actions(current_state)
            if not possible_actions:
                return None

            for action in possible_actions:
                next_state = self.game.get_next_state(current_state, action)
                next_state_tuple = tuple(map(tuple, next_state.puzzle))

                cost = Heuristics.misplaced_tiles(next_state)
                cost += next_state.cost

                if next_state_tuple not in visited_states:
                    heapq.heappush(priority_queue, (cost, next_state))
                    steps_data.append({
                        "misplaced_tiles": cost,
                        "state": next_state.puzzle,
                        "action": action,
                        "parent": current_state.puzzle
                    })

        steps_data.insert(0, {
            "misplaced_tiles": Heuristics.misplaced_tiles(initial_state),
            "state": initial_state.puzzle,
            "action": None,
            "parent": None
        })

        return None

In [2]:
import random

def randomise_4x4_matrix():
    """
    Randomizes a 4x4 matrix.
    """
    values = list(range(16))
    random.shuffle(values)
    matrix = [values[i:i + 4] for i in range(0, 16, 4)]
    return matrix

def solve_puzzle(search, initial_state):
    """
    Solves the puzzle based on the specified search algorithm.
    """
    game = Game()
    agent = Agent(game)
    if search == "astar_manhattan":
        solution = agent.solve_Astar_manhattan(State(initial_state))
    elif search == "astar_misplaced":
        solution = agent.solve_Astar_misplaced(State(initial_state))
    elif search == "BFS":
        solution = agent.solve_BFS(State(initial_state))
    return solution

def print_matrix(matrix, init=1):
    """
    Prints the matrix in a readable format.
    """
    for state in matrix[init:]:
        if init == 0:
            print(" ".join(map(str, state)))
        else:
            for row in state:
                print(" ".join(map(str, row)))
            print()

def reconstruct_path(solution):
    """
    Reconstructs the path based on the solution.
    """
    path = [solution[-1]['state']]
    parent_state = solution[-1]['parent']

    while parent_state:
        if parent_state == solution[0]['state']:
            path.insert(0, parent_state)
            break
        parent_index = None
        for i, step in enumerate(solution):
            if step['state'] == parent_state:
                parent_index = i
                break

        if parent_index is not None:
            path.insert(0, parent_state)
            parent_state = solution[parent_index]['parent']
        else:
            break

    return path


n_attempts = 1
successful = 0
goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]


while successful < n_attempts:
    initial_state = [[1, 2, 3, 4],
                     [5, 6, 7, 8],
                     [9, 10, 12, 15],
                     [13, 14, 11, 0]]  # Example initial state

    solution_bfs = solve_puzzle("BFS", initial_state)
    solution_astar_manhattan = solve_puzzle("astar_manhattan", initial_state)
    solution_astar_misplaced = solve_puzzle("astar_misplaced", initial_state)

    last_manhattan = []
    last_misplaced = []

    if (
        solution_bfs is not None and len(solution_bfs) > 0 and
        solution_astar_manhattan is not None and
        solution_astar_misplaced is not None
    ):
        print("Initial State\n")
        print_matrix(initial_state, init=0)

        print("\nNumber of nodes explored with Breadth-First Search (BFS)   :", sum(item[1] for item in solution_bfs))
        print("Number of nodes explored with A* with Manhattan Distance heuristic:", len(solution_astar_manhattan) - 1)
        print("Number of nodes explored with A* with Misplaced Tiles heuristic   :", len(solution_astar_misplaced) - 1)
        print()

        last_manhattan = [solution_astar_manhattan, len(solution_astar_manhattan) - 1]
        last_misplaced = [solution_astar_misplaced, len(solution_astar_misplaced) - 1]

        successful += 1
    else:
        print("No solution found.")
        break


Initial State

1 2 3 4
5 6 7 8
9 10 12 15
13 14 11 0

Number of nodes explored with Breadth-First Search (BFS)   : 41
Number of nodes explored with A* with Manhattan Distance heuristic: 9
Number of nodes explored with A* with Misplaced Tiles heuristic   : 12



In [3]:
for i, j in solution_bfs:
    print(i, j)

1 1
2 2
3 4
4 10
5 24


In [4]:
a = reconstruct_path(solution_astar_manhattan)
print(len(a))

5


In [5]:
b = reconstruct_path(solution_astar_misplaced)
print(len(b))

5


In [6]:
b

[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]]

In [7]:
a

[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]],
 [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]]

In [8]:
a == b

True